## Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import json
from pprint import pprint
import requests
from bs4 import BeautifulSoup
from functools import reduce
from pandas import json_normalize
import os 

## Loading the City_bike csv

In [2]:
city_bike = pd.read_csv('city_bike.csv')
city_bike

,empty_slots,free_bikes,id,latitude,longitude,bike_station,number_of_bikes
0,2,17,fb337bbed72e2be090071e199899b2be,43.665269,-79.319796,Queen St E / Woodward Ave,19
1,11,2,4ff88d5880e71aa40d34cfe5d09b0ca7,43.671420,-79.445947,Primrose Ave / Davenport Rd,13
2,2,19,a09c67c0b419654d907c9134b108e328,43.666224,-79.317693,Queen St E / Rhodes Ave,21
3,26,9,d6a9daee68070a8b106cfb598d81308c,43.653236,-79.376716,Bond St / Queen St E,35
4,9,15,8f8af40d9388c8a3962559e8681d3db7,43.663722,-79.380288,Church St / Alexander St,24
...,...,...,...,...,...,...,...
772,9,8,bf5d4d14e7619f5ff51e264e4095cc4b,43.650298,-79.366485,Princess St / Front St E,17
773,23,0,c53d12128be1f1022c4b78518cca7675,43.662453,-79.397838,St. George St / Willcocks St,23
774,23,2,d0f019574616cb2bce86924059a4c0f8,43.647043,-79.383141,York St / Wellington St W (2),25
775,4,10,479d21ebdac47eb8e30a209268e0bebe,43.644887,-79.456412,High Park Blvd / Parkside Dr,14


## Basic Response from API

In [3]:
# Replace with your Yelp API key
yelp_key = os.environ['YELP_KEY']

In [4]:
def post_request(lat, long, categories, radius):
    yelp_key 
    url = f"https://api.yelp.com/v3/businesses/search?term=delis&latitude={lat}&longitude={long}"
    headers = {
    "Authorization": f"Bearer {yelp_key}"
    }
    response = requests.get(url, headers=headers)
    data = response.json()
    #pprint(data)
    return data

In [5]:
"""I had to use only 100 records because the data to be extracted was to larger for my system to handle.
I had to literally interupt the kernel."""

city_bike2 = city_bike.head(100)

## For Loop to iterate through the various lat and long in the city_bike dataframe.

In [6]:
lst = []
for index, row in city_bike2.iterrows():
    #print(row['latitude'], row['longitude'])
    response = post_request(row['latitude'], row['longitude'],'businesses',1000)
    lst.append(response)

## Creating a DataFrame

In [7]:
yelp_df = pd.DataFrame()
for i in lst:
    df1 = pd.DataFrame(i['businesses'])
    yelp_df = pd.concat([yelp_df,df1])

In [8]:
yelp_df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,xWlb7E10Ysb8PxM0eciecA,oak-park-deli-east-york,Oak Park Deli,https://s3-media1.fl.yelpcdn.com/bphoto/8GFg5c...,False,https://www.yelp.com/biz/oak-park-deli-east-yo...,24,"[{'alias': 'delis', 'title': 'Delis'}, {'alias...",4.5,"{'latitude': 43.69388, 'longitude': -79.31004}",[],$,"{'address1': '213 Oak Park Avenue', 'address2'...",+14164243354,+1 416-424-3354,3275.758631
1,IVnLsgz_qo8NWVhR09SF3Q,east-york-deli-toronto,East York Deli,https://s3-media1.fl.yelpcdn.com/bphoto/RuJtTH...,False,https://www.yelp.com/biz/east-york-deli-toront...,2,"[{'alias': 'delis', 'title': 'Delis'}, {'alias...",5.0,"{'latitude': 43.69404, 'longitude': -79.3276}",[],NaN,"{'address1': '903 Coxwell Ave', 'address2': ''...",+14164293663,+1 416-429-3663,3263.043138
2,VmIWtyejlvSOIDoBGztNag,casa-mia-delicatessen-and-fine-foods-toronto,Casa Mia Delicatessen & Fine Foods,https://s3-media2.fl.yelpcdn.com/bphoto/owaoDK...,False,https://www.yelp.com/biz/casa-mia-delicatessen...,6,"[{'alias': 'cheese', 'title': 'Cheese Shops'}]",3.5,"{'latitude': 43.6703271666323, 'longitude': -7...",[],NaN,"{'address1': '1985 Queen Street E', 'address2'...",+16473491340,+1 647-349-1340,1578.842913
3,ptmG_KjwGVszU0sPpFGXJQ,schefflers-delicatessen-toronto,Scheffler's Delicatessen,https://s3-media4.fl.yelpcdn.com/bphoto/p-iIDa...,False,https://www.yelp.com/biz/schefflers-delicatess...,26,"[{'alias': 'delis', 'title': 'Delis'}, {'alias...",4.5,"{'latitude': 43.64914, 'longitude': -79.37157}",[],$$,"{'address1': '93 Front St E', 'address2': '', ...",+14163642806,+1 416-364-2806,4553.128807
4,0bWa6gq8KST_Gopiy5klcA,plank-road-market-toronto,Plank Road Market,https://s3-media3.fl.yelpcdn.com/bphoto/f-PyC5...,False,https://www.yelp.com/biz/plank-road-market-tor...,8,"[{'alias': 'grocery', 'title': 'Grocery'}]",4.5,"{'latitude': 43.68436, 'longitude': -79.32021}",[],$,"{'address1': '1716 Danforth Avenue', 'address2...",+14164635474,+1 416-463-5474,2114.618564
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15,I5QklsBP4IWTbfeJFoGGZQ,kingsway-meat-products-and-deli-toronto,Kingsway Meat Products & Deli,https://s3-media4.fl.yelpcdn.com/bphoto/Y1eiJf...,False,https://www.yelp.com/biz/kingsway-meat-product...,15,"[{'alias': 'delicatessen', 'title': 'Delicates...",4.0,"{'latitude': 43.6503299, 'longitude': -79.48093}",[],$$,"{'address1': '2342 Bloor St W', 'address2': ''...",+14167625365,+1 416-762-5365,3853.657544
16,UNnYvWgV7D32mHWQfBg2mQ,georges-deli-and-bbq-toronto-2,George's Deli & BBQ,https://s3-media3.fl.yelpcdn.com/bphoto/nmS65x...,False,https://www.yelp.com/biz/georges-deli-and-bbq-...,29,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",4.0,"{'latitude': 43.66493, 'longitude': -79.4109}",[],$,"{'address1': '795 Bathurst Street', 'address2'...",+14165368494,+1 416-536-8494,2096.808388
17,29zc-Uw40X7LnH8jBMSWuw,nosso-talho-toronto-2,Nosso Talho,https://s3-media3.fl.yelpcdn.com/bphoto/D310mL...,False,https://www.yelp.com/biz/nosso-talho-toronto-2...,11,"[{'alias': 'meats', 'title': 'Meat Shops'}]",3.0,"{'latitude': 43.64964, 'longitude': -79.42641}",[],$,"{'address1': '1326 Dundas St W', 'address2': '...",+14165330101,+1 416-533-0101,877.289935
18,Bj2g9Kgij_yOn5HQRt0BnQ,liberty-village-market-and-cafe-toronto,Liberty Village Market & Cafe,https://s3-media4.fl.yelpcdn.com/bphoto/muhGyc...,False,https://www.yelp.com/biz/liberty-village-marke...,21,"[{'alias': 'convenience', 'title': 'Convenienc...",4.0,"{'latitude': 43.63796, 'longitude': -79.42131}",[],$,"{'address1': '65 Jefferson Avenue', 'address2'...",+14165300477,+1 416-530-0477,2203.267028


### Dropping unwanted columns

In [9]:
yelp_df.drop(columns = ['alias','image_url','url','phone','price'],axis = 1, inplace = True)
yelp_df

,id,name,is_closed,review_count,categories,rating,coordinates,transactions,location,display_phone,distance
0,xWlb7E10Ysb8PxM0eciecA,Oak Park Deli,False,24,"[{'alias': 'delis', 'title': 'Delis'}, {'alias...",4.5,"{'latitude': 43.69388, 'longitude': -79.31004}",[],"{'address1': '213 Oak Park Avenue', 'address2'...",+1 416-424-3354,3275.758631
1,IVnLsgz_qo8NWVhR09SF3Q,East York Deli,False,2,"[{'alias': 'delis', 'title': 'Delis'}, {'alias...",5.0,"{'latitude': 43.69404, 'longitude': -79.3276}",[],"{'address1': '903 Coxwell Ave', 'address2': ''...",+1 416-429-3663,3263.043138
2,VmIWtyejlvSOIDoBGztNag,Casa Mia Delicatessen & Fine Foods,False,6,"[{'alias': 'cheese', 'title': 'Cheese Shops'}]",3.5,"{'latitude': 43.6703271666323, 'longitude': -7...",[],"{'address1': '1985 Queen Street E', 'address2'...",+1 647-349-1340,1578.842913
3,ptmG_KjwGVszU0sPpFGXJQ,Scheffler's Delicatessen,False,26,"[{'alias': 'delis', 'title': 'Delis'}, {'alias...",4.5,"{'latitude': 43.64914, 'longitude': -79.37157}",[],"{'address1': '93 Front St E', 'address2': '', ...",+1 416-364-2806,4553.128807
4,0bWa6gq8KST_Gopiy5klcA,Plank Road Market,False,8,"[{'alias': 'grocery', 'title': 'Grocery'}]",4.5,"{'latitude': 43.68436, 'longitude': -79.32021}",[],"{'address1': '1716 Danforth Avenue', 'address2...",+1 416-463-5474,2114.618564
...,...,...,...,...,...,...,...,...,...,...,...
15,I5QklsBP4IWTbfeJFoGGZQ,Kingsway Meat Products & Deli,False,15,"[{'alias': 'delicatessen', 'title': 'Delicates...",4.0,"{'latitude': 43.6503299, 'longitude': -79.48093}",[],"{'address1': '2342 Bloor St W', 'address2': ''...",+1 416-762-5365,3853.657544
16,UNnYvWgV7D32mHWQfBg2mQ,George's Deli & BBQ,False,29,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",4.0,"{'latitude': 43.66493, 'longitude': -79.4109}",[],"{'address1': '795 Bathurst Street', 'address2'...",+1 416-536-8494,2096.808388
17,29zc-Uw40X7LnH8jBMSWuw,Nosso Talho,False,11,"[{'alias': 'meats', 'title': 'Meat Shops'}]",3.0,"{'latitude': 43.64964, 'longitude': -79.42641}",[],"{'address1': '1326 Dundas St W', 'address2': '...",+1 416-533-0101,877.289935
18,Bj2g9Kgij_yOn5HQRt0BnQ,Liberty Village Market & Cafe,False,21,"[{'alias': 'convenience', 'title': 'Convenienc...",4.0,"{'latitude': 43.63796, 'longitude': -79.42131}",[],"{'address1': '65 Jefferson Avenue', 'address2'...",+1 416-530-0477,2203.267028


## Data Cleaning Process

Some of the columns in the Dataframe had a list of nested dictionary,to extract the data need i used the json_normalize( ) function.

In [10]:
yelp_df_normalized = json_normalize(yelp_df['categories'])
yelp_df_normalized

,0,1,2
0,"{'alias': 'delis', 'title': 'Delis'}","{'alias': 'sandwiches', 'title': 'Sandwiches'}",None
1,"{'alias': 'delis', 'title': 'Delis'}","{'alias': 'breakfast_brunch', 'title': 'Breakf...","{'alias': 'sandwiches', 'title': 'Sandwiches'}"
2,"{'alias': 'cheese', 'title': 'Cheese Shops'}",None,None
3,"{'alias': 'delis', 'title': 'Delis'}","{'alias': 'cheese', 'title': 'Cheese Shops'}",None
4,"{'alias': 'grocery', 'title': 'Grocery'}",None,None
...,...,...,...
1995,"{'alias': 'delicatessen', 'title': 'Delicatess...","{'alias': 'meats', 'title': 'Meat Shops'}",None
1996,"{'alias': 'sandwiches', 'title': 'Sandwiches'}","{'alias': 'burgers', 'title': 'Burgers'}",None
1997,"{'alias': 'meats', 'title': 'Meat Shops'}",None,None
1998,"{'alias': 'convenience', 'title': 'Convenience...","{'alias': 'tradamerican', 'title': 'American'}",None


In [11]:
yelp_df_normalized.drop(columns = [1,2],axis = 1, inplace = True)
yelp_df_normalized

,0
0,"{'alias': 'delis', 'title': 'Delis'}"
1,"{'alias': 'delis', 'title': 'Delis'}"
2,"{'alias': 'cheese', 'title': 'Cheese Shops'}"
3,"{'alias': 'delis', 'title': 'Delis'}"
4,"{'alias': 'grocery', 'title': 'Grocery'}"
...,...
1995,"{'alias': 'delicatessen', 'title': 'Delicatess..."
1996,"{'alias': 'sandwiches', 'title': 'Sandwiches'}"
1997,"{'alias': 'meats', 'title': 'Meat Shops'}"
1998,"{'alias': 'convenience', 'title': 'Convenience..."


In [12]:
yelp_df_normalized = json_normalize(yelp_df_normalized[0])
yelp_df_normalized

,alias,title
0,delis,Delis
1,delis,Delis
2,cheese,Cheese Shops
3,delis,Delis
4,grocery,Grocery
...,...,...
1995,delicatessen,Delicatessen
1996,sandwiches,Sandwiches
1997,meats,Meat Shops
1998,convenience,Convenience Stores


In [13]:
yelp_df_normalized.drop('alias',axis = 1, inplace = True)
yelp_df_normalized

,title
0,Delis
1,Delis
2,Cheese Shops
3,Delis
4,Grocery
...,...
1995,Delicatessen
1996,Sandwiches
1997,Meat Shops
1998,Convenience Stores


In [14]:
yelp_df['categories'] = yelp_df_normalized
yelp_df

,id,name,is_closed,review_count,categories,rating,coordinates,transactions,location,display_phone,distance
0,xWlb7E10Ysb8PxM0eciecA,Oak Park Deli,False,24,Delis,4.5,"{'latitude': 43.69388, 'longitude': -79.31004}",[],"{'address1': '213 Oak Park Avenue', 'address2'...",+1 416-424-3354,3275.758631
1,IVnLsgz_qo8NWVhR09SF3Q,East York Deli,False,2,Delis,5.0,"{'latitude': 43.69404, 'longitude': -79.3276}",[],"{'address1': '903 Coxwell Ave', 'address2': ''...",+1 416-429-3663,3263.043138
2,VmIWtyejlvSOIDoBGztNag,Casa Mia Delicatessen & Fine Foods,False,6,Cheese Shops,3.5,"{'latitude': 43.6703271666323, 'longitude': -7...",[],"{'address1': '1985 Queen Street E', 'address2'...",+1 647-349-1340,1578.842913
3,ptmG_KjwGVszU0sPpFGXJQ,Scheffler's Delicatessen,False,26,Delis,4.5,"{'latitude': 43.64914, 'longitude': -79.37157}",[],"{'address1': '93 Front St E', 'address2': '', ...",+1 416-364-2806,4553.128807
4,0bWa6gq8KST_Gopiy5klcA,Plank Road Market,False,8,Grocery,4.5,"{'latitude': 43.68436, 'longitude': -79.32021}",[],"{'address1': '1716 Danforth Avenue', 'address2...",+1 416-463-5474,2114.618564
...,...,...,...,...,...,...,...,...,...,...,...
15,I5QklsBP4IWTbfeJFoGGZQ,Kingsway Meat Products & Deli,False,15,Grocery,4.0,"{'latitude': 43.6503299, 'longitude': -79.48093}",[],"{'address1': '2342 Bloor St W', 'address2': ''...",+1 416-762-5365,3853.657544
16,UNnYvWgV7D32mHWQfBg2mQ,George's Deli & BBQ,False,29,Diners,4.0,"{'latitude': 43.66493, 'longitude': -79.4109}",[],"{'address1': '795 Bathurst Street', 'address2'...",+1 416-536-8494,2096.808388
17,29zc-Uw40X7LnH8jBMSWuw,Nosso Talho,False,11,Meat Shops,3.0,"{'latitude': 43.64964, 'longitude': -79.42641}",[],"{'address1': '1326 Dundas St W', 'address2': '...",+1 416-533-0101,877.289935
18,Bj2g9Kgij_yOn5HQRt0BnQ,Liberty Village Market & Cafe,False,21,Delis,4.0,"{'latitude': 43.63796, 'longitude': -79.42131}",[],"{'address1': '65 Jefferson Avenue', 'address2'...",+1 416-530-0477,2203.267028


In [15]:
yelp_df1_normalized = json_normalize(yelp_df['coordinates'])
yelp_df1_normalized

,latitude,longitude
0,43.693880,-79.310040
1,43.694040,-79.327600
2,43.670327,-79.301455
3,43.649140,-79.371570
4,43.684360,-79.320210
...,...,...
1995,43.650330,-79.480930
1996,43.664930,-79.410900
1997,43.649640,-79.426410
1998,43.637960,-79.421310


In [16]:
# Replacing the content of some columns in the DataFrame.

yelp_df['latitude'] = yelp_df1_normalized['latitude']
yelp_df['longitude']= yelp_df1_normalized['longitude']
yelp_df

,id,name,is_closed,review_count,categories,rating,coordinates,transactions,location,display_phone,distance,latitude,longitude
0,xWlb7E10Ysb8PxM0eciecA,Oak Park Deli,False,24,Delis,4.5,"{'latitude': 43.69388, 'longitude': -79.31004}",[],"{'address1': '213 Oak Park Avenue', 'address2'...",+1 416-424-3354,3275.758631,43.693880,-79.310040
1,IVnLsgz_qo8NWVhR09SF3Q,East York Deli,False,2,Delis,5.0,"{'latitude': 43.69404, 'longitude': -79.3276}",[],"{'address1': '903 Coxwell Ave', 'address2': ''...",+1 416-429-3663,3263.043138,43.694040,-79.327600
2,VmIWtyejlvSOIDoBGztNag,Casa Mia Delicatessen & Fine Foods,False,6,Cheese Shops,3.5,"{'latitude': 43.6703271666323, 'longitude': -7...",[],"{'address1': '1985 Queen Street E', 'address2'...",+1 647-349-1340,1578.842913,43.670327,-79.301455
3,ptmG_KjwGVszU0sPpFGXJQ,Scheffler's Delicatessen,False,26,Delis,4.5,"{'latitude': 43.64914, 'longitude': -79.37157}",[],"{'address1': '93 Front St E', 'address2': '', ...",+1 416-364-2806,4553.128807,43.649140,-79.371570
4,0bWa6gq8KST_Gopiy5klcA,Plank Road Market,False,8,Grocery,4.5,"{'latitude': 43.68436, 'longitude': -79.32021}",[],"{'address1': '1716 Danforth Avenue', 'address2...",+1 416-463-5474,2114.618564,43.684360,-79.320210
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15,I5QklsBP4IWTbfeJFoGGZQ,Kingsway Meat Products & Deli,False,15,Grocery,4.0,"{'latitude': 43.6503299, 'longitude': -79.48093}",[],"{'address1': '2342 Bloor St W', 'address2': ''...",+1 416-762-5365,3853.657544,43.680693,-79.286773
16,UNnYvWgV7D32mHWQfBg2mQ,George's Deli & BBQ,False,29,Diners,4.0,"{'latitude': 43.66493, 'longitude': -79.4109}",[],"{'address1': '795 Bathurst Street', 'address2'...",+1 416-536-8494,2096.808388,43.647147,-79.396239
17,29zc-Uw40X7LnH8jBMSWuw,Nosso Talho,False,11,Meat Shops,3.0,"{'latitude': 43.64964, 'longitude': -79.42641}",[],"{'address1': '1326 Dundas St W', 'address2': '...",+1 416-533-0101,877.289935,43.685490,-79.314990
18,Bj2g9Kgij_yOn5HQRt0BnQ,Liberty Village Market & Cafe,False,21,Delis,4.0,"{'latitude': 43.63796, 'longitude': -79.42131}",[],"{'address1': '65 Jefferson Avenue', 'address2'...",+1 416-530-0477,2203.267028,43.648182,-79.379485


In [17]:
yelp_df2_normalized = json_normalize(yelp_df['location'])
yelp_df2_normalized

,address1,address2,address3,city,zip_code,country,state,display_address
0,213 Oak Park Avenue,,,East York,M4C 4N2,CA,ON,"[213 Oak Park Avenue, East York, ON M4C 4N2, C..."
1,903 Coxwell Ave,,None,Toronto,M4C 2T9,CA,ON,"[903 Coxwell Ave, Toronto, ON M4C 2T9, Canada]"
2,1985 Queen Street E,,None,Toronto,M4L 1J1,CA,ON,"[1985 Queen Street E, Toronto, ON M4L 1J1, Can..."
3,93 Front St E,,,Toronto,M5E 1C3,CA,ON,"[93 Front St E, Toronto, ON M5E 1C3, Canada]"
4,1716 Danforth Avenue,,,Toronto,M4C 1H8,CA,ON,"[1716 Danforth Avenue, Toronto, ON M4C 1H8, Ca..."
...,...,...,...,...,...,...,...,...
1995,2342 Bloor St W,,,Toronto,M6S 1P3,CA,ON,"[2342 Bloor St W, Toronto, ON M6S 1P3, Canada]"
1996,795 Bathurst Street,,,Toronto,M5S 1Z5,CA,ON,"[795 Bathurst Street, Toronto, ON M5S 1Z5, Can..."
1997,1326 Dundas St W,,,Toronto,M6J 1Y1,CA,ON,"[1326 Dundas St W, Toronto, ON M6J 1Y1, Canada]"
1998,65 Jefferson Avenue,,,Toronto,M6K 1Y3,CA,ON,"[65 Jefferson Avenue, Toronto, ON M6K 1Y3, Can..."


In [18]:
# Creating an address variable from the various columns above
address = yelp_df2_normalized['address1']+','+ yelp_df2_normalized['city']+','+ yelp_df2_normalized['zip_code']+','+yelp_df2_normalized['state']+',' + yelp_df2_normalized['country']
address

0       213 Oak Park Avenue,East York,M4C 4N2,ON,CA
1             903 Coxwell Ave,Toronto,M4C 2T9,ON,CA
2         1985 Queen Street E,Toronto,M4L 1J1,ON,CA
3               93 Front St E,Toronto,M5E 1C3,ON,CA
4        1716 Danforth Avenue,Toronto,M4C 1H8,ON,CA
                           ...                     
1995          2342 Bloor St W,Toronto,M6S 1P3,ON,CA
1996      795 Bathurst Street,Toronto,M5S 1Z5,ON,CA
1997         1326 Dundas St W,Toronto,M6J 1Y1,ON,CA
1998      65 Jefferson Avenue,Toronto,M6K 1Y3,ON,CA
1999          7 Camden Street,Toronto,M5V 1V2,ON,CA
Length: 2000, dtype: object

In [19]:
# Creating a new column in the DataFrame 
yelp_df['location'] = address
yelp_df

,id,name,is_closed,review_count,categories,rating,coordinates,transactions,location,display_phone,distance,latitude,longitude
0,xWlb7E10Ysb8PxM0eciecA,Oak Park Deli,False,24,Delis,4.5,"{'latitude': 43.69388, 'longitude': -79.31004}",[],"213 Oak Park Avenue,East York,M4C 4N2,ON,CA",+1 416-424-3354,3275.758631,43.693880,-79.310040
1,IVnLsgz_qo8NWVhR09SF3Q,East York Deli,False,2,Delis,5.0,"{'latitude': 43.69404, 'longitude': -79.3276}",[],"903 Coxwell Ave,Toronto,M4C 2T9,ON,CA",+1 416-429-3663,3263.043138,43.694040,-79.327600
2,VmIWtyejlvSOIDoBGztNag,Casa Mia Delicatessen & Fine Foods,False,6,Cheese Shops,3.5,"{'latitude': 43.6703271666323, 'longitude': -7...",[],"1985 Queen Street E,Toronto,M4L 1J1,ON,CA",+1 647-349-1340,1578.842913,43.670327,-79.301455
3,ptmG_KjwGVszU0sPpFGXJQ,Scheffler's Delicatessen,False,26,Delis,4.5,"{'latitude': 43.64914, 'longitude': -79.37157}",[],"93 Front St E,Toronto,M5E 1C3,ON,CA",+1 416-364-2806,4553.128807,43.649140,-79.371570
4,0bWa6gq8KST_Gopiy5klcA,Plank Road Market,False,8,Grocery,4.5,"{'latitude': 43.68436, 'longitude': -79.32021}",[],"1716 Danforth Avenue,Toronto,M4C 1H8,ON,CA",+1 416-463-5474,2114.618564,43.684360,-79.320210
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15,I5QklsBP4IWTbfeJFoGGZQ,Kingsway Meat Products & Deli,False,15,Grocery,4.0,"{'latitude': 43.6503299, 'longitude': -79.48093}",[],"976 Kingston Road,Toronto,M4E 1S9,ON,CA",+1 416-762-5365,3853.657544,43.680693,-79.286773
16,UNnYvWgV7D32mHWQfBg2mQ,George's Deli & BBQ,False,29,Diners,4.0,"{'latitude': 43.66493, 'longitude': -79.4109}",[],"7 Camden Street,Toronto,M5V 1V2,ON,CA",+1 416-536-8494,2096.808388,43.647147,-79.396239
17,29zc-Uw40X7LnH8jBMSWuw,Nosso Talho,False,11,Meat Shops,3.0,"{'latitude': 43.64964, 'longitude': -79.42641}",[],"1968 Danforth Avenue,Toronto,M4C 1J6,ON,CA",+1 416-533-0101,877.289935,43.685490,-79.314990
18,Bj2g9Kgij_yOn5HQRt0BnQ,Liberty Village Market & Cafe,False,21,Delis,4.0,"{'latitude': 43.63796, 'longitude': -79.42131}",[],"199 Bay Street,Toronto,M5L 1G5,ON,CA",+1 416-530-0477,2203.267028,43.648182,-79.379485


In [20]:
# Dropping unwanted coulmns in the DataFrame

yelp_df.drop(columns = ['coordinates', 'transactions'], axis = 1, inplace = True)
yelp_df

,id,name,is_closed,review_count,categories,rating,location,display_phone,distance,latitude,longitude
0,xWlb7E10Ysb8PxM0eciecA,Oak Park Deli,False,24,Delis,4.5,"213 Oak Park Avenue,East York,M4C 4N2,ON,CA",+1 416-424-3354,3275.758631,43.693880,-79.310040
1,IVnLsgz_qo8NWVhR09SF3Q,East York Deli,False,2,Delis,5.0,"903 Coxwell Ave,Toronto,M4C 2T9,ON,CA",+1 416-429-3663,3263.043138,43.694040,-79.327600
2,VmIWtyejlvSOIDoBGztNag,Casa Mia Delicatessen & Fine Foods,False,6,Cheese Shops,3.5,"1985 Queen Street E,Toronto,M4L 1J1,ON,CA",+1 647-349-1340,1578.842913,43.670327,-79.301455
3,ptmG_KjwGVszU0sPpFGXJQ,Scheffler's Delicatessen,False,26,Delis,4.5,"93 Front St E,Toronto,M5E 1C3,ON,CA",+1 416-364-2806,4553.128807,43.649140,-79.371570
4,0bWa6gq8KST_Gopiy5klcA,Plank Road Market,False,8,Grocery,4.5,"1716 Danforth Avenue,Toronto,M4C 1H8,ON,CA",+1 416-463-5474,2114.618564,43.684360,-79.320210
...,...,...,...,...,...,...,...,...,...,...,...
15,I5QklsBP4IWTbfeJFoGGZQ,Kingsway Meat Products & Deli,False,15,Grocery,4.0,"976 Kingston Road,Toronto,M4E 1S9,ON,CA",+1 416-762-5365,3853.657544,43.680693,-79.286773
16,UNnYvWgV7D32mHWQfBg2mQ,George's Deli & BBQ,False,29,Diners,4.0,"7 Camden Street,Toronto,M5V 1V2,ON,CA",+1 416-536-8494,2096.808388,43.647147,-79.396239
17,29zc-Uw40X7LnH8jBMSWuw,Nosso Talho,False,11,Meat Shops,3.0,"1968 Danforth Avenue,Toronto,M4C 1J6,ON,CA",+1 416-533-0101,877.289935,43.685490,-79.314990
18,Bj2g9Kgij_yOn5HQRt0BnQ,Liberty Village Market & Cafe,False,21,Delis,4.0,"199 Bay Street,Toronto,M5L 1G5,ON,CA",+1 416-530-0477,2203.267028,43.648182,-79.379485


In [21]:
# Saving DataFrame to csv

yelp_df.to_csv('yelp_df.csv',index = False)